In [2]:
import torch, torchaudio, torchvision.transforms as transforms, matplotlib.pyplot as plt, torch.nn as nn, torch.optim as optim, numpy as np
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import  StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, auc, classification_report, roc_auc_score
from torch.autograd import grad

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

data = np.load("../hvcm/RFQ.npy", allow_pickle=True)
label = np.load("../hvcm/RFQ_labels.npy", allow_pickle=True)
label = label[:, 1]  # Assuming the second column is the label
label = (label == "Fault").astype(int)  # Convert to binary labels
print(data.shape, label.shape)

normal_indices = np.where(label == 0)

cuda
NVIDIA A100-PCIE-40GB
(872, 4500, 14) (872,)


# Time GAN

## Model Components 

In [3]:
class RNNBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout=0.0):
        super().__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        h, _ = self.rnn(x)
        return self.linear(h)

class Embedder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.block = RNNBlock(input_dim, hidden_dim, num_layers)

    def forward(self, x):
        return self.block(x)

class Recovery(nn.Module):
    def __init__(self, hidden_dim, input_dim, num_layers):
        super().__init__()
        self.block = RNNBlock(hidden_dim, input_dim, num_layers)

    def forward(self, h):
        return self.block(h)

class Generator(nn.Module):
    def __init__(self, z_dim, hidden_dim, num_layers):
        super().__init__()
        self.block = RNNBlock(z_dim, hidden_dim, num_layers)

    def forward(self, z):
        return self.block(z)

class Supervisor(nn.Module):
    def __init__(self, hidden_dim, num_layers):
        super().__init__()
        self.block = RNNBlock(hidden_dim, hidden_dim, num_layers)

    def forward(self, h):
        return self.block(h)

class Discriminator(nn.Module):
    def __init__(self, hidden_dim, num_layers):
        super().__init__()
        self.block = RNNBlock(hidden_dim, 1, num_layers)

    def forward(self, h):
        return self.block(h)


## Loss Functions

In [4]:
import torch
import torch.nn.functional as F

def reconstruction_loss(x, x_tilde):
    return F.mse_loss(x_tilde, x)

def supervised_loss(h, h_hat_supervise):
    return F.mse_loss(h_hat_supervise[:, :-1, :], h[:, 1:, :])

def generator_loss(y_fake, h_hat_supervise, x, x_hat):
    g_loss_u = torch.mean(torch.nn.functional.binary_cross_entropy_with_logits(y_fake, torch.ones_like(y_fake)))
    g_loss_s = supervised_loss(h_hat_supervise, h_hat_supervise)
    g_loss_v = reconstruction_loss(x, x_hat)
    return g_loss_u + g_loss_s + 100 * torch.sqrt(g_loss_v)

def discriminator_loss(y_real, y_fake):
    d_loss_real = F.binary_cross_entropy_with_logits(y_real, torch.ones_like(y_real))
    d_loss_fake = F.binary_cross_entropy_with_logits(y_fake, torch.zeros_like(y_fake))
    return d_loss_real + d_loss_fake


## Training

In [ ]:
def train_timegan(data, model_dict, optimizer_dict, epochs):
    embedder, recovery, generator, supervisor, discriminator = model_dict.values()
    e_opt, g_opt, d_opt = optimizer_dict.values()

    for epoch in range(epochs):
        # Phase 1: Embedder training
        h = embedder(data)
        x_tilde = recovery(h)
        e_loss = reconstruction_loss(data, x_tilde)
        e_opt.zero_grad()
        e_loss.backward()
        e_opt.step()

        # Phase 2: Supervisor training
        h = embedder(data)
        h_hat_supervise = supervisor(h)
        s_loss = supervised_loss(h, h_hat_supervise)
        g_opt.zero_grad()
        s_loss.backward()
        g_opt.step()

        # Phase 3: Joint training
        z = torch.randn_like(data)
        e_h = embedder(data)
        h_hat = generator(z)
        h_hat_supervise = supervisor(h_hat)
        x_hat = recovery(h_hat_supervise)
        y_fake = discriminator(h_hat_supervise)
        y_real = discriminator(e_h)

        g_loss = generator_loss(y_fake, h_hat_supervise, data, x_hat)
        d_loss = discriminator_loss(y_real, y_fake)

        g_opt.zero_grad()
        g_loss.backward()
        g_opt.step()

        d_opt.zero_grad()
        d_loss.backward()
        d_opt.step()

        print(f"[{epoch}] E_loss: {e_loss.item():.4f}, G_loss: {g_loss.item():.4f}, D_loss: {d_loss.item():.4f}")


In [ ]:
input_dim = 24
hidden_dim = 32
num_layers = 3
epochs = 1000

embedder = Embedder(input_dim, hidden_dim, num_layers)
recovery = Recovery(hidden_dim, input_dim, num_layers)
generator = Generator(input_dim, hidden_dim, num_layers)
supervisor = Supervisor(hidden_dim, num_layers)
discriminator = Discriminator(hidden_dim, num_layers)

model_dict = {
    'embedder': embedder,
    'recovery': recovery,
    'generator': generator,
    'supervisor': supervisor,
    'discriminator': discriminator
}

optimizer_dict = {
    'embedder': torch.optim.Adam(embedder.parameters(), lr=1e-3),
    'generator': torch.optim.Adam(list(generator.parameters()) + list(supervisor.parameters()), lr=1e-3),
    'discriminator': torch.optim.Adam(discriminator.parameters(), lr=1e-3)
}

# Dummy data for illustration
data = torch.rand((64, 24, input_dim))
print(data.shape)
train_timegan(data, model_dict, optimizer_dict, epochs)


RuntimeError: input.size(-1) must be equal to input_size. Expected 32, got 24

# Generate and Combine

In [ ]:
# combine_data = np.concatenate((generated_samples, data), axis=0)  # Combine real and generated data
# combine_labels = np.concatenate((np.zeros(num_samples), label), axis=0)  # Labels: 0 for real, 0 for generated

# Processing: Mel Spec > Resizing > Feature Extraction

In [ ]:
# Resize and convert to 3-channel image
def resize_spectrogram(spectrogram):
    spectrogram = (spectrogram - spectrogram.min()) / (spectrogram.max() - spectrogram.min() + 1e-6)
    spectrogram = np.uint8(spectrogram.cpu().numpy() * 255)
    spectrogram = np.stack([spectrogram] * 3, axis=-1)
    image = Image.fromarray(spectrogram)
    image = transforms.Resize((224, 224))(image)
    return transforms.ToTensor()(image)

# Process dataset
def process_dataset(data):
    num_samples, _, num_channels = data.shape
    features = np.zeros((num_samples, num_channels, 4096))
    mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=2500000, n_mels=128).to(device)
    model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).to(device)
    model.classifier = model.classifier[:-3]
    model.eval()

    for i in range(num_samples):
        for j in range(num_channels):
            ts = torch.tensor(data[i, :, j], dtype=torch.float32).to(device)
            mel = mel_transform(ts)
            img = resize_spectrogram(mel)
            with torch.no_grad():
                feat = model(img.unsqueeze(0).to(device))
            features[i, j, :] = feat.squeeze().cpu().numpy()
    return features

# Mel Scale comparison

# AE Class

In [ ]:
# Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size=4096):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64), 
            nn.ReLU(),
            nn.Linear(64, 32), 
            nn.ReLU(),
            nn.Linear(32, 16), 
            nn.ReLU(),
            nn.Linear(16, 8), 
            nn.ReLU(),
            nn.Linear(8, 4), 
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 16), 
            nn.ReLU(),
            nn.Linear(16, 32), 
            nn.ReLU(),
            nn.Linear(32, 64), 
            nn.ReLU(),
            nn.Linear(64, input_size), 
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))


# Train autoencoder
def train_autoencoder(features, epochs=20, batch_size=128):
    x = torch.tensor(features.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader = DataLoader(TensorDataset(x), batch_size=batch_size, shuffle=True)
    model = Autoencoder().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            inputs = batch[0]
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(loader):.6f}")
    return model


def print_eval(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))
  print("F1 = {}".format(f1_score(labels, predictions)))
  print(confusion_matrix(labels, predictions))

# Plot reconstruction error histogram
def plot_reconstruction_error(model, features, percentile=95):
    x = torch.tensor(features.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader = DataLoader(TensorDataset(x), batch_size=64)
    errors = []
    criterion = nn.MSELoss(reduction='none')

    with torch.no_grad():
        for batch in loader:
            inputs = batch[0]
            outputs = model(inputs)
            batch_errors = criterion(outputs, inputs).mean(dim=1)
            errors.extend(batch_errors.cpu().numpy())

    threshold = np.percentile(errors, percentile)
    anomalies = np.sum(np.array(errors) > threshold)

    plt.hist(errors, bins=50, alpha=0.75)
    plt.axvline(threshold, color='r', linestyle='--', label=f'Threshold ({percentile}%)')
    plt.xlabel('Reconstruction Error')
    plt.ylabel('Frequency')
    plt.title('Reconstruction Error Histogram')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"Anomaly threshold: {threshold:.6f}")
    print(f"Detected anomalies: {anomalies}")


# Cross Validation without Scalers

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
features = process_dataset(combine_data)
normal_indices = np.where(combine_labels == 0)[0]
print("Features shape:", features.shape)
for fold, (train_idx, val_idx) in enumerate(skf.split(features, combine_labels)):
    print(f"Fold {fold + 1}")
    train_fold_data, val_fold_data = features[train_idx], features[val_idx]
    train_fold_labels, val_fold_labels = combine_labels[train_idx], combine_labels[val_idx]

    # Train autoencoder on the training fold
    model = train_autoencoder(features[normal_indices], epochs=15, batch_size=64)

    # Evaluate on validation fold
    x_val = torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader_val = DataLoader(TensorDataset(x_val), batch_size=64)
    
    # Compute reconstruction errors
    x = model(torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)).cpu().detach().numpy()
    errors = np.mean((x - val_fold_data.reshape(-1, 4096)) ** 2, axis=1)

    # Reshape to (175, 14)
    errors = errors.reshape(val_fold_data.shape[0], val_fold_data.shape[1])

    # Aggregate per sample (e.g., mean across channels)
    sample_errors = np.mean(errors, axis=1)

    percentile = 90
    # Thresholding
    threshold = np.percentile(sample_errors, percentile)
    predictions = (sample_errors > threshold).astype(int)


    
    plot_reconstruction_error(model, val_fold_data, percentile=percentile)
    print_eval(predictions, val_fold_labels)

# Cross Validation with StandardScaler

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scaled_data = StandardScaler().fit_transform(combine_data.reshape(-1, data.shape[-1])).reshape(combine_data.shape)
features = process_dataset(scaled_data)
normal_indices = np.where(combine_labels == 0)[0]
print("Features shape:", features.shape)
for fold, (train_idx, val_idx) in enumerate(skf.split(features, combine_labels)):
    print(f"Fold {fold + 1}")
    train_fold_data, val_fold_data = features[train_idx], features[val_idx]
    train_fold_labels, val_fold_labels = combine_labels[train_idx], combine_labels[val_idx]

    # Train autoencoder on the training fold
    model = train_autoencoder(features[normal_indices], epochs=15, batch_size=64)

    # Evaluate on validation fold
    x_val = torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader_val = DataLoader(TensorDataset(x_val), batch_size=64)
    
    # Compute reconstruction errors
    x = model(torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)).cpu().detach().numpy()
    errors = np.mean((x - val_fold_data.reshape(-1, 4096)) ** 2, axis=1)

    # Reshape to (175, 14)
    errors = errors.reshape(val_fold_data.shape[0], val_fold_data.shape[1])

    # Aggregate per sample (e.g., mean across channels)
    sample_errors = np.mean(errors, axis=1)

    percentile = 90
    # Thresholding
    threshold = np.percentile(sample_errors, percentile)
    predictions = (sample_errors > threshold).astype(int)


    
    plot_reconstruction_error(model, val_fold_data, percentile=percentile)
    print_eval(predictions, val_fold_labels)

# Cross Validation with MinMax

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scaled_data = MinMaxScaler().fit_transform(combine_data.reshape(-1, combine_data.shape[-1])).reshape(combine_data.shape)
features = process_dataset(scaled_data)
normal_indices = np.where(combine_labels == 0)[0]
print("Features shape:", features.shape)
for fold, (train_idx, val_idx) in enumerate(skf.split(features, combine_labels)):
    print(f"Fold {fold + 1}")
    train_fold_data, val_fold_data = features[train_idx], features[val_idx]
    train_fold_labels, val_fold_labels = combine_labels[train_idx], combine_labels[val_idx]

    # Train autoencoder on the training fold
    model = train_autoencoder(features[normal_indices], epochs=15, batch_size=64)

    # Evaluate on validation fold
    x_val = torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader_val = DataLoader(TensorDataset(x_val), batch_size=64)
    
    # Compute reconstruction errors
    x = model(torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)).cpu().detach().numpy()
    errors = np.mean((x - val_fold_data.reshape(-1, 4096)) ** 2, axis=1)

    # Reshape to (175, 14)
    errors = errors.reshape(val_fold_data.shape[0], val_fold_data.shape[1])

    # Aggregate per sample (e.g., mean across channels)
    sample_errors = np.mean(errors, axis=1)

    percentile = 90
    # Thresholding
    threshold = np.percentile(sample_errors, percentile)
    predictions = (sample_errors > threshold).astype(int)


    
    plot_reconstruction_error(model, val_fold_data, percentile=percentile)
    print_eval(predictions, val_fold_labels)

# Observation:
Comparing with and without normalizing data 

### MinMaxed scored

Accuracy = 0.8461538461538461

Precision = 0.3125

Recall = 0.2777777777777778

F1 = 0.29411764705882354

[[254  22]

[ 26  10]]

---

### StandardScaled scored


Accuracy = 0.782051282051282

Precision = 0.0

Recall = 0.0

F1 = 0.0

[[244  32]

[ 36   0]]


---

### Without any normlaization scored

Accuracy = 0.782051282051282

Precision = 0.0

Recall = 0.0

F1 = 0.0

[[244  32]
 
[ 36   0]]
